# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine 
import numpy as np
import pandas as pd

# To use regex
# Importing re
import re

# To Process Strings 
#import nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DB_messeges.db')
df = pd.read_sql_table("T_messeges", engine)
X = df.iloc[:,0]
Y = df.iloc[:,3:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    """This function normalize the string, remove punctuations, and then create a list of tokens. 
    Args:
        text: a string 
    Returns:
        processed_list : a processed list of strings (normalized, punctuations free, and tokenized)
    """
    
    # Convert string to lowercase
    text = text.lower()
    
    # Remove punctuations
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # Tokenize words
    tokenized_words = word_tokenize(text)
    
    stemer = SnowballStemmer("english")
    processed_list = [stemer.stem(token) for token in tokenized_words if token not in stopwords.words("english")]
    
    return processed_list    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 12)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
pred_y =pipeline.predict(X= X_test)

In [7]:
for i in np.arange(0,35):
    print(Y_test.columns[i],classification_report(y_pred= pred_y[:,i], y_true= Y_test.iloc[:,i]))

related              precision    recall  f1-score   support

          0       0.62      0.49      0.55      1516
          1       0.86      0.91      0.88      4991

avg / total       0.80      0.81      0.80      6507

request              precision    recall  f1-score   support

          0       0.89      0.98      0.93      5410
          1       0.79      0.42      0.55      1097

avg / total       0.87      0.88      0.87      6507

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6479
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6507

aid_related              precision    recall  f1-score   support

          0       0.76      0.85      0.80      3867
          1       0.74      0.61      0.67      2640

avg / total       0.75      0.75      0.75      6507

medical_help              precision    recall  f1-score   support

          0       0.93      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {'vect__min_df': [1, 3],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[6,10], 
              'clf__estimator__min_samples_split':[4,8]}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
cv.fit(X_train, Y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv_pred_y =pipeline.predict(X= X_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
for i in np.arange(0,35):
    print(Y_test.columns[i],classification_report(y_pred= pred_y[:,i], y_true= Y_test.iloc[:,i]))

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.